In [ ]:
# %% [code]
# ================================
# 1) ENV REPAIR & DEPENDENCIES
#    (Run ONCE, then Restart the kernel)
# ================================
!pip -q install -U "protobuf<5" "pyarrow>=14,<20" \
  "transformers==4.44.2" "datasets==2.19.0" \
  "accelerate==1.0.1" "evaluate==0.4.2" \
  "sentencepiece==0.2.0" "scikit-learn==1.5.2"

import sys, subprocess
print("Python:", sys.version)
subprocess.run([
    "python", "-c",
    "import google.protobuf, pyarrow as pa, transformers, datasets, sklearn, evaluate, sentencepiece;"
    "import google.protobuf; print('protobuf:', google.protobuf.__version__);"
    "print('pyarrow:', pa.__version__);"
    "print('transformers:', transformers.__version__);"
    "print('datasets:', datasets.__version__);"
    "import sklearn; print('sklearn:', sklearn.__version__);"
    "import evaluate; print('evaluate:', evaluate.__version__);"
    "import sentencepiece; import torch; import platform;"
    "print('torch:', torch.__version__); print('platform:', platform.platform())"
])
print("\nNow go to Runtime → Restart. Then continue with the next cell.")


In [1]:
# %% [code]
# ================================
# 2) IMPORTS & CONFIG
# ================================
import os
import json
import random
import math
import numpy as np
import pandas as pd

import torch

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import evaluate

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    set_seed,
)

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
set_seed(SEED)

# Model (compact multilingual; practical alternative to "DistilXLM-RoBERTa-base")
MODEL_NAME = "distilbert/distilbert-base-multilingual-cased"

# Where Kaggle might place your JSON
CANDIDATE_PATHS = [
    "/kaggle/input/train-data/train_data.json"    # e.g., Dataset named 'train_data'
]

DATA_PATH = next((p for p in CANDIDATE_PATHS if os.path.exists(p)), None)
assert DATA_PATH is not None, f"Couldn't find train_data.json in {CANDIDATE_PATHS}. Add it as a Kaggle input."

print("Using data:", DATA_PATH)
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))


2025-11-13 05:37:16.914968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763012236.937845     125 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763012236.944719     125 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using data: /kaggle/input/train-data/train_data.json
GPU available: True
CUDA device: Tesla P100-PCIE-16GB


In [2]:
# %% [code]
# ================================
# 3) LOAD & PREP DATA
# ================================
with open(DATA_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Keep columns we need; predict 'validity' from 'syllogism'
needed = ["syllogism", "validity"]
missing = [c for c in needed if c not in df.columns]
assert not missing, f"Missing columns in JSON: {missing}"

df = df[needed].dropna().reset_index(drop=True)
df["label"] = df["validity"].astype(int)
df = df.rename(columns={"syllogism": "text"})

print(df.head(3))
print("\nLabel distribution (0=invalid, 1=valid):")
print(df["label"].value_counts(normalize=True).round(3))


                                                text  validity  label
0  All cars are a type of vehicle. No animal is a...     False      0
1  Nothing that is a soda is a juice. A portion o...      True      1
2  Everything that is a planet is a celestial bod...     False      0

Label distribution (0=invalid, 1=valid):
label
0    0.5
1    0.5
Name: proportion, dtype: float64


In [3]:
# %% [code]
# ================================
# 4) TRAIN / VALIDATION SPLIT
# ================================
train_df, val_df = train_test_split(
    df[["text", "label"]],
    test_size=0.15,
    random_state=SEED,
    stratify=df["label"]
)

ds = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df.reset_index(drop=True)),
})

len(ds["train"]), len(ds["validation"])


(816, 144)

In [4]:
# %% [code]
# ================================
# 5) TOKENIZATION
# ================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def tok_fn(batch):
    return tokenizer(batch["text"], truncation=True)

cols_to_remove = [c for c in ds["train"].column_names if c not in ["text", "label"]]
ds_tok = ds.map(tok_fn, batched=True, remove_columns=cols_to_remove)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print(ds_tok)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/816 [00:00<?, ? examples/s]

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 816
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 144
    })
})


In [5]:
# %% [code]
# ================================
# 6) METRICS  (updated to match screenshot keys)
# ================================
import evaluate
accuracy_metric  = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric    = evaluate.load("recall")
f1_metric        = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy":  accuracy_metric.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision_metric.compute(predictions=preds, references=labels, average="weighted")["precision"],
        "recall":    recall_metric.compute(predictions=preds, references=labels, average="weighted")["recall"],
        "f1":        f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }


In [8]:
# %% [code]
# ================================
# 7) MODEL & TRAINING ARGS  (GPU-HEAVY)
# ================================
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels
)

EPOCHS =  5           
BSZ = 16
LR = 2e-5
WD = 0.01
GRAD_ACCUM = 1
LOG_STEPS = 50

args = TrainingArguments(
    output_dir="outputs",
    learning_rate=LR,
    per_device_train_batch_size=BSZ,
    per_device_eval_batch_size=BSZ,
    gradient_accumulation_steps=GRAD_ACCUM,
    num_train_epochs=EPOCHS,
    weight_decay=WD,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",   # ← track best by F1 (matches summary cell)
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
    logging_steps=LOG_STEPS,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

train_result = trainer.train()
train_result.metrics


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.662100,0.588391,0.687500,0.701534,0.687500,0.681963
2,0.508200,0.563485,0.715278,0.720423,0.715278,0.713607
3,0.373400,0.557983,0.729167,0.739564,0.729167,0.726196
4,0.305700,0.549362,0.777778,0.779720,0.777778,0.777391
5,0.248300,0.543782,0.791667,0.791892,0.791667,0.791626


{'train_runtime': 51.5252,
 'train_samples_per_second': 79.185,
 'train_steps_per_second': 4.949,
 'total_flos': 47162533527744.0,
 'train_loss': 0.41638573291254977,
 'epoch': 5.0}

In [10]:
# %% [code]
# ================================
# 8) EVALUATE (GPU-OK but fast)
# ================================
metrics = trainer.evaluate()
metrics


{'eval_loss': 0.5437819957733154,
 'eval_accuracy': 0.7916666666666666,
 'eval_precision': 0.791891891891892,
 'eval_recall': 0.7916666666666666,
 'eval_f1': 0.7916264711557013,
 'eval_runtime': 0.1655,
 'eval_samples_per_second': 870.267,
 'eval_steps_per_second': 54.392,
 'epoch': 5.0}

In [11]:
# %% [code]
# ================================
# 8b) SUMMARY PRINTER + BEST EPOCH
# ================================
from pprint import pprint

# Grab all eval logs (one per epoch) from HF Trainer state
eval_logs = [e for e in trainer.state.log_history if isinstance(e, dict) and "eval_accuracy" in e]

if len(eval_logs) == 0:
    # Fallback: run a fresh eval now
    current = trainer.evaluate()
    print(current)
    best_epoch = None
    best_log = None
else:
    # Print the final (last) eval dict like your top line
    last_eval = eval_logs[-1]
    pprint(last_eval)
    print("\nEvaluating on validation data...")
    # Run one immediate eval (mirrors your second line in screenshot)
    current = trainer.evaluate()
    pprint(current)

    # Final Validation Summary block
    print("\nFinal Validation Summary:")
    print(f"  Accuracy:  {current.get('eval_accuracy', 0):.4f}")
    print(f"  F1 Score:  {current.get('eval_f1', 0):.4f}")
    print(f"  Precision: {current.get('eval_precision', 0):.4f}")
    print(f"  Recall:    {current.get('eval_recall', 0):.4f}")

    # Determine best epoch by highest eval_f1 among eval logs
    best_log = max(eval_logs, key=lambda d: d.get("eval_f1", float('-inf')))
    best_epoch = best_log.get("epoch", None)

# Also show best checkpoint (Trainer tracked)
best_ckpt = getattr(trainer.state, "best_model_checkpoint", None)

if best_epoch is not None:
    print("\nBest Epoch Summary (by F1):")
    print(f"  Best epoch: {best_epoch}")
    print(f"  Accuracy:   {best_log.get('eval_accuracy', 0):.4f}")
    print(f"  F1 Score:   {best_log.get('eval_f1', 0):.4f}")
    print(f"  Precision:  {best_log.get('eval_precision', 0):.4f}")
    print(f"  Recall:     {best_log.get('eval_recall', 0):.4f}")
if best_ckpt:
    print(f"  Best checkpoint: {best_ckpt}")


{'epoch': 5.0,
 'eval_accuracy': 0.7916666666666666,
 'eval_f1': 0.7916264711557013,
 'eval_loss': 0.5437819957733154,
 'eval_precision': 0.791891891891892,
 'eval_recall': 0.7916666666666666,
 'eval_runtime': 0.1655,
 'eval_samples_per_second': 870.267,
 'eval_steps_per_second': 54.392,
 'step': 255}

Evaluating on validation data...
{'epoch': 5.0,
 'eval_accuracy': 0.7916666666666666,
 'eval_f1': 0.7916264711557013,
 'eval_loss': 0.5437819957733154,
 'eval_precision': 0.791891891891892,
 'eval_recall': 0.7916666666666666,
 'eval_runtime': 0.1667,
 'eval_samples_per_second': 864.048,
 'eval_steps_per_second': 54.003}

Final Validation Summary:
  Accuracy:  0.7917
  F1 Score:  0.7916
  Precision: 0.7919
  Recall:    0.7917

Best Epoch Summary (by F1):
  Best epoch: 5.0
  Accuracy:   0.7917
  F1 Score:   0.7916
  Precision:  0.7919
  Recall:     0.7917
  Best checkpoint: outputs/checkpoint-255


In [12]:
# %% [code]
# ================================
# 9) SAVE MODEL + TOKENIZER
# ================================
SAVE_DIR = "model_validity_distilmbert_multilingual"
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("Saved to:", SAVE_DIR)


Saved to: model_validity_distilmbert_multilingual


In [13]:
# %% [code]
# ================================
# 10) QUICK INFERENCE DEMO (GPU-ACCELERATED IF AVAILABLE)
# ================================
ex_texts = [
    "All mammals are animals. All dogs are mammals. Therefore, all dogs are animals.",
    "No birds are mammals. Some whales are mammals. Therefore, some whales are birds.",
]
enc = tokenizer(ex_texts, padding=True, truncation=True, return_tensors="pt").to(model.device)
with torch.no_grad():
    logits = model(**enc).logits
preds = logits.argmax(dim=1).cpu().tolist()

for t, p in zip(ex_texts, preds):
    print(f"[pred={p}] {t}")
print("\n(1 = formally valid, 0 = not valid)")


[pred=0] All mammals are animals. All dogs are mammals. Therefore, all dogs are animals.
[pred=0] No birds are mammals. Some whales are mammals. Therefore, some whales are birds.

(1 = formally valid, 0 = not valid)
